https://www.tensorflow.org/tutorials/load_data/csv

In [20]:


from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

SyntaxError: invalid syntax (<ipython-input-20-fb4876345b8a>, line 1)

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)


16384/13049 [=====================================] - 0s 1us/step


In [4]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)


In [5]:
!head {train_file_path}


survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [6]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]


In [7]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)


Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [8]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))


In [9]:
show_batch(raw_train_data)


sex                 : [b'male' b'male' b'female' b'male' b'female']
age                 : [11. 28. 42. 50. 15.]
n_siblings_spouses  : [1 0 0 1 0]
parch               : [2 0 0 0 1]
fare                : [120.      7.25   13.    106.425 211.337]
class               : [b'First' b'Third' b'Second' b'First' b'First']
deck                : [b'B' b'unknown' b'unknown' b'C' b'B']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'n' b'y' b'y' b'n' b'n']


In [10]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)


sex                 : [b'male' b'female' b'female' b'male' b'female']
age                 : [24. 45.  2. 28. 58.]
n_siblings_spouses  : [0 1 0 1 0]
parch               : [0 1 1 0 0]
fare                : [  7.142 164.867  12.288  15.5   146.521]
class               : [b'Third' b'First' b'Third' b'Third' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'B']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Queenstown' b'Cherbourg']
alone               : [b'y' b'n' b'n' b'n' b'y']


In [11]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)


age                 : [18. 59. 60. 28.  4.]
n_siblings_spouses  : [1 0 1 0 0]
class               : [b'Third' b'Third' b'Second' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'A' b'unknown']
alone               : [b'n' b'y' b'n' b'y' b'n']


In [12]:
show_batch(raw_train_data)


sex                 : [b'male' b'male' b'female' b'male' b'male']
age                 : [39. 28. 63. 40. 28.]
n_siblings_spouses  : [0 0 0 0 8]
parch               : [0 0 0 0 2]
fare                : [ 0.     7.75   9.587  7.896 69.55 ]
class               : [b'First' b'Third' b'Third' b'Third' b'Third']
deck                : [b'A' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Queenstown' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'n']


In [13]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [14]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [16]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))
show_batch(packed_train_data)


sex                 : [b'female' b'female' b'male' b'male' b'male']
class               : [b'Third' b'Third' b'Third' b'Second' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Queenstown' b'Southampton' b'Queenstown' b'Southampton' b'Southampton']
alone               : [b'y' b'n' b'n' b'y' b'y']
numeric             : [[28.     0.     0.     7.733]
 [18.     0.     1.     9.35 ]
 [ 4.     4.     1.    29.125]
 [31.     0.     0.    13.   ]
 [47.     0.     0.     9.   ]]


In [17]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [18]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [19]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])


In [21]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [22]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f88241a4c80>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [23]:
example_batch['numeric']


<tf.Tensor: id=499, shape=(5, 4), dtype=float32, numpy=
array([[ 14.   ,   5.   ,   2.   ,  46.9  ],
       [ 64.   ,   1.   ,   4.   , 263.   ],
       [ 30.   ,   0.   ,   0.   , 106.425],
       [ 49.   ,   1.   ,   1.   , 110.883],
       [ 40.5  ,   0.   ,   0.   ,   7.75 ]], dtype=float32)>

In [24]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()


array([[-1.249,  3.87 ,  2.043,  0.229],
       [ 2.747,  0.395,  4.565,  4.187],
       [ 0.029, -0.474, -0.479,  1.319],
       [ 1.548,  0.395,  0.782,  1.401],
       [ 0.869, -0.474, -0.479, -0.488]], dtype=float32)

In [25]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [26]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))


In [27]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [28]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])


Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [29]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)


In [30]:
print(preprocessing_layer(example_batch).numpy()[0])


[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -1.249  3.87
  2.043  0.229  1.     0.   ]


In [31]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])


In [32]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [33]:
model.fit(train_data, epochs=20)


Epoch 1/20
126/126 [==============================] - 2s 20ms/step - loss: 0.5100 - accuracy: 0.7592
Epoch 2/20
126/126 [==============================] - 1s 10ms/step - loss: 0.4171 - accuracy: 0.8246
Epoch 3/20
126/126 [==============================] - 1s 11ms/step - loss: 0.4069 - accuracy: 0.8341
Epoch 4/20
126/126 [==============================] - 1s 11ms/step - loss: 0.3929 - accuracy: 0.8405
Epoch 5/20
126/126 [==============================] - 1s 11ms/step - loss: 0.3828 - accuracy: 0.8453
Epoch 6/20
126/126 [==============================] - 1s 11ms/step - loss: 0.3730 - accuracy: 0.8501
Epoch 7/20
126/126 [==============================] - 2s 14ms/step - loss: 0.3705 - accuracy: 0.8421
Epoch 8/20
126/126 [==============================] - 1s 10ms/step - loss: 0.3564 - accuracy: 0.8501
Epoch 9/20
126/126 [==============================] - 1s 11ms/step - loss: 0.3489 - accuracy: 0.8501
Epoch 10/20
126/126 [==============================] - 1s 11ms/step - loss: 0.3522 - accura

In [34]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))


53/53 [==============================] - 1s 14ms/step - loss: 0.4782 - accuracy: 0.8182


Test Loss 0.47816299408111934, Test Accuracy 0.8181818127632141


In [35]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))


Predicted survival: 96.94%  | Actual outcome:  DIED
Predicted survival: 29.16%  | Actual outcome:  DIED
Predicted survival: 79.70%  | Actual outcome:  DIED
Predicted survival: 89.88%  | Actual outcome:  DIED
Predicted survival: 32.75%  | Actual outcome:  SURVIVED
